In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import folium


In [3]:
df = pd.read_csv('dataset/train.csv')

RENAME COLUNS

In [4]:
df = df.rename(columns={'Classificação do hotel': 'hotel_rating', 'Meses da reserva até o check-in': 'month_booking',
                        'Número de pernoites reservadas': 'number_stays_booking', 'Número de hospedes': 'number_guests',
                        'Regime de alimentação': 'meal_regime', 'Nacionalidade': 'nationality', 'Forma de Reserva': 'reservation_Form',
                        'Já se hospedou anterioremente': 'previously_hosted', 'Tipo do quarto reservado': 'type_booked_room',
                        'Reserva feita por agência de turismo': 'tourism_agency_booking', 'Reserva feita por empresa': 'company_booking',
                        'Reserva com Estacionamento': 'parking_Reservation', 'Reserva com Observações': 'reservation_Observations', 'Reserva Cancelada': 'booking_canceled'})

CLEAN NAN

In [5]:
df['nationality'] = df['nationality'].apply(
    lambda x: 'nao_identificado' if pd.isnull(x) else x)

df['number_guests'] = df['number_guests'].apply(
    lambda x: 1 if pd.isnull(x) else x)

CLEAN DUPLICAT

In [6]:
df.duplicated().sum()
df

,id,hotel_rating,month_booking,number_stays_booking,number_guests,meal_regime,nationality,reservation_Form,previously_hosted,type_booked_room,tourism_agency_booking,company_booking,parking_Reservation,reservation_Observations,booking_canceled
0,33571,5 estrelas,5,5,2.0,Café da manha e jantar,France,Agência,Não,Amethyst,Sim,Não,Sim,Nenhuma,0
1,82458,4 estrelas,167,3,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
2,94061,4 estrelas,4,3,2.0,Café da manha,Belgium,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0
3,75196,4 estrelas,13,2,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
4,82940,4 estrelas,4,2,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72154,101851,4 estrelas,2,3,2.0,Café da manha,France,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,0
72155,50800,4 estrelas,4,2,2.0,Sem refeicao,Italy,Agência,Não,Amethyst,Sim,Não,Sim,1 a 3,0
72156,4101,5 estrelas,5,2,2.0,Café da manha e jantar,Spain,Agência,Não,Amethyst,Não,Não,Não,Nenhuma,1
72157,15828,5 estrelas,1,8,2.0,Café da manha e jantar,Spain,Balcão,Não,Pink Sapphire,Sim,Não,Não,Nenhuma,0


# filter canceled and not canceled


In [7]:
def filter_canceled(df):
    linhas = df['booking_canceled'] == 1
    df = df.loc[linhas, :]
    return df

def filter_Nocanceled(df):
    linhas = df['booking_canceled'] == 0
    df = df.loc[linhas, :]
    return df

# 1 - número de id com booking canceled por tipo de meal_regime

In [8]:
df_02 = df.copy()
linhas = df_02['booking_canceled'] == 1
df_02 = df_02.loc[linhas, :]
df_aux = df_02.loc[:, ['id', 'meal_regime']].groupby('meal_regime').count().reset_index()
px.pie(df_aux, values='id', names='meal_regime')

In [9]:
df_02 = df.copy()
linhas = df_02['booking_canceled'] == 0
df_02 = df_02.loc[linhas, :]
df_aux = df_02.loc[:, ['id', 'meal_regime']].groupby('meal_regime').count().reset_index()
px.pie(df_aux, values='id', names='meal_regime')

anotações: maioria dos pedidos tem preferencia por café da manhã

# 2 - número de cancelamento e não cancelamento

In [10]:
df_01 = df.copy()
df_01 = df_01.loc[:, ['id', 'booking_canceled']].groupby('booking_canceled').count().reset_index()
px.pie(df_01, values='id', names='booking_canceled')

# 3 - Cancelamento por forma de reserva

In [11]:
df_01 = filter_canceled(df)
df_01 = df_01.loc[:, ['id', 'reservation_Form']].groupby('reservation_Form').count().reset_index()
px.pie(df_01, values='id', names='reservation_Form')


# 4 - Não cancelamento por forma de reserva

In [12]:
df_01 = filter_Nocanceled(df)
df_01 = df_01.loc[:, ['id', 'reservation_Form']].groupby('reservation_Form').count().reset_index()
px.pie(df_01, values='id', names='reservation_Form')

# 5 - número de cancelamento por quantidade de reservas 

In [13]:
df_aux = filter_canceled(df)
df_aux = (df_aux.loc[:, ['id', 'number_stays_booking']]
                .groupby('number_stays_booking')
                .count()
                .sort_values('id', ascending=False)
                .reset_index()
                .head(10))

px.pie(df_aux, values='id', names='number_stays_booking')



# 6 - número de não cancelamento por quantidade de reservas 

In [14]:
df_aux = filter_Nocanceled(df)
df_aux = (df_aux.loc[:, ['id', 'number_stays_booking']]
                .groupby('number_stays_booking')
                .count()
                .sort_values('id', ascending=False)
                .reset_index()
                .head(10))

px.pie(df_aux, values='id', names='number_stays_booking')

# 7 - número de cancelamento em relação aos dias para realização do check-in

In [15]:
df_aux = filter_canceled(df)
df_aux = (df_aux.loc[:, ['id', 'month_booking']]
                .groupby('month_booking')
                .count()
                .sort_values('id', ascending=False)
                .reset_index()
                .head(10))

px.pie(df_aux, values='id', names='month_booking')


# 8 - número de cancelamento por tipo de quarto.

In [17]:
df_aux = filter_canceled(df)
df_aux = (df_aux.loc[:, ['id', 'type_booked_room']]
                .groupby('type_booked_room')
                .count()
                .sort_values('id', ascending=False)
                .reset_index()
                .head(10))

px.pie(df_aux, values='id', names='type_booked_room')